In [1]:
import laspy
import matplotlib.pyplot as plt
import numpy as np
import open3d
from PIL import Image, ImageFont, ImageDraw
import os
from pathlib import Path

from open3d.web_visualizer import draw

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [2]:
# root path for dataset(lidar files and label files)
LIDAR_ROOT = '/nas2/YJ/git/tree_data_raw/pcdet/data/tree/training/velodyne/'
LABEL_ROOT = '/nas2/YJ/git/tree_data_raw/pcdet/data/tree/training/labels/'

# txt file of indices that you want to check
INDICES_FILE = '/nas2/YJ/git/tree_data_raw/pcdet/data/tree/ImageSets/train.txt'

# path for saving result file
SAVE_PATH = '/nas2/YJ/tree_lidar/train/'

In [3]:
SAVE_PATH = Path(SAVE_PATH)
SAVE_PATH.mkdir(parents=True, exist_ok=True)

In [4]:
assert os.path.exists(LIDAR_ROOT)
assert os.path.exists(LABEL_ROOT)
assert os.path.exists(INDICES_FILE)

# Main

In [5]:
def get_points_and_objects(index) :
    las_path = os.path.join(LIDAR_ROOT, 'T{}.las'.format(index))

    lasfile = laspy.file.File(str(las_path), mode="r")
    las_points = np.vstack((lasfile.x -30.0 , lasfile.y -30.0
                                     , lasfile.z-300.0)).transpose().astype(np.float32)
    
    label_file = os.path.join(LABEL_ROOT, '{}.txt'.format(index))

    objects = []
    with open(label_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            label = line.strip().split(' ')
            h = float(label[8])
            l = float(label[9])
            w = float(label[10])
            loc = np.array((float(label[11]), float(label[12]), float(label[13])), dtype=np.float32)
            gt = [loc[0]-30.0, loc[1]-30.0, loc[2]-300.0, l, w, h, 0]
            objects.append(gt)    
            
    return las_points, objects

In [6]:
with open(INDICES_FILE) as f :
    indices = f.readlines()
    
for index in indices :
    index = int(index)
    las_points, objects = get_points_and_objects(index)
    
    ## Visualize
    vis = open3d.visualization.Visualizer()
    vis.create_window()
    
    vis.get_render_option().point_size = 1.0
    vis.get_render_option().background_color = np.zeros(3)

    axis_pcd = open3d.geometry.TriangleMesh.create_coordinate_frame(size=10.0, origin=[0, 0, 0])
    vis.add_geometry(axis_pcd)

    pts = open3d.geometry.PointCloud()
    pts.points = open3d.utility.Vector3dVector(las_points)

    vis.add_geometry(pts)
    pts.colors = open3d.utility.Vector3dVector(np.ones((las_points.shape[0], 3)))

    #### Adding GT boxes ####
    for gt_boxes in objects :
        center = gt_boxes[0:3]
        lwh = gt_boxes[3:6]
        axis_angles = np.array([0, 0, 1e-100])
        rot = open3d.geometry.get_rotation_matrix_from_axis_angle(axis_angles)
        box3d = open3d.geometry.OrientedBoundingBox(center, rot, lwh)

        line_set = open3d.geometry.LineSet.create_from_oriented_bounding_box(box3d)

        lines = np.asarray(line_set.lines)
        lines = np.concatenate([lines, np.array([[1, 4], [7, 6]])], axis=0)

        line_set.lines = open3d.utility.Vector2iVector(lines)
        line_set.paint_uniform_color((0,1,0))
        vis.add_geometry(line_set)  

    ### Adding File name (Sometimes it doesn't work well in my desktop, so I leave the belows as comments)
    #FONT = '/usr/share/fonts/truetype/dejavu/DejaVuSansMono-Bold.ttf'
    #img = Image.new('RGB', (1920, 1080), color = (0,0,0))
    #fnt = ImageFont.truetype(FONT, 64)
    #d = ImageDraw.Draw(img)
    #filename = 'T{}.las'.format(index)
    #d.text((20,20), filename, font=fnt, fill=(255,255,255)) # puts text in upper left
    #img.save('tmp.png')
    #im = open3d.io.read_image("./tmp.png")
    #vis.add_geometry(im)
    
    vis.run()
    # Save
    vis.capture_screen_image(str(SAVE_PATH / '{}.png'.format(index)))

vis.destroy_window()